# In C++
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/yggdrasil-decision-forests/blob/main/documentation/public/docs/tutorial/cpp.ipynb)


## Setup

In [ ]:
pip install ydf -U

## Serving in C++
YDF models can be served directly in C++ through the C++ library. Since the Python API and the C++ API share the same serving code, models are fully cross-compatible.

**Benefits of serving with C++**
-   Optimized inference speed: The C++ API offers full control over the serving code, which can be used to squeeze every nanosecond of performance out of YDF.
-   Optimized binary size: Since the C++ serving code does not depend on the training code, only a small fraction of YDF must be linked.

**When not to use the C++ API**
-   The C++ API is not as easy to use as the Python API.
-   Preprocessing, if any, must be regenerated in C++.


## Training a small model

The next cell creates a very small YDF model.

In [2]:
# Load libraries
import ydf  # Yggdrasil Decision Forests
import pandas as pd  # We use Pandas to load small datasets

# Download a classification dataset and load it as a Pandas DataFrame.
ds_path = "https://raw.githubusercontent.com/google/yggdrasil-decision-forests/main/yggdrasil_decision_forests/test_data/dataset"
train_ds = pd.read_csv(f"{ds_path}/iris.csv")
label = "class"

model = ydf.RandomForestLearner(label=label, num_trees=10).train(train_ds)

model.describe()

Train model on 150 examples
Model trained in 0:00:00.003721


## Generate the C++ code

With `model.to_cpp()`, YDF creates a working C++ file that can be imported in an existing C++ project. The namespace of the C++ code is controlled with the `key=` parameter.

In [3]:
# Save the model code to model.h and display it
with open("ydf_tutorial_model.h", "w") as f:
  f.write(model.to_cpp(key="ydf_tutorial"))

!cat ydf_tutorial_model.h

// Automatically generated code running an Yggdrasil Decision Forests model in
// C++. This code was generated with "model.to_cpp()".
//
// Date of generation: 2023-11-01 13:06:59.075973
// YDF Version: 0.0.3
//
// How to use this code:
//
// 1. Copy this code in a new .h file.
// 2. If you use Bazel/Blaze, use the following dependencies:
//      //third_party/absl/status:statusor
//      //third_party/absl/strings
//      //external/ydf_cc/yggdrasil_decision_forests/api:serving
// 3. In your existing code, include the .h file and do:
//   // Load the model (to do only once).
//   namespace ydf = yggdrasil_decision_forests;
//   const auto model = ydf::exported_model_123::Load(<path to model>);
//   // Run the model
//   predictions = model.Predict();
// 4. By default, the "Predict" function takes no inputs and creates fake
//   examples. In practice, you want to add your input data as arguments to
//   "Predict" and call "examples->Set..." functions accordingly.
// 4. (Bonus)
//   All

## Using the C++ code.

To use the C++ code in a project, follow these steps.

1.  If you use Bazel/Blaze, create a rule with the dependencies:
```
      //third_party/absl/status:statusor,
      //third_party/absl/strings,
      //third_party/yggdrasil_decision_forests/api:serving,
```
1.  In your C++ code, include the .h file and call the model with:
```
    // Load the model (to do only once).
    namespace ydf = yggdrasil_decision_forests;
    const auto model = ydf::exported_model_ydf_tutorial::Load(<path to model>);
    // Run the model
    predictions = model.Predict();
```
1.  The generated "Predict" function takes no inputs. Instead, it fills the
    input features with placeholder values. Therefore, you will want to add
    your input as arguments to the "Predict" function, and use it to populate
    the "examples->Set..." section accordingly.

## Further improvements

You can further optimize the inference speed by pre-allocating and reusing the examples and predictions for each thread running the model. 